In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import math
import copy
from collections import deque
import random
from IPython.display import clear_output

In [ ]:
# we start by putting in place the structure of our code

In [ ]:
# create_folds.py

In [ ]:
import time
import copy
import torch
from torch import nn
from torch.nn import functional as F
# import chainer
# import chainer.functions as F
# import chainer.links as L

In [ ]:
device = 'cuda'

In [ ]:
len_embed = 30
num_prod_to_rec = 12 # number of products to recommend to a user at each time step
beta = 0.5
scale = 100
# episode_length = 128

In [ ]:
transactions_train = '../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
df_transactions = pd.read_csv(transactions_train,
                             dtype= {
                                 'customer_id': 'str',
                                 'article_id': 'str'
                             })
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])
df_transactions = df_transactions.set_index('t_dat')
print(df_transactions.index.min(), df_transactions.index.max())
df_transactions.head()

In [ ]:
df_transactions.info()

In [ ]:
customers = '../input/h-and-m-personalized-fashion-recommendations/customers.csv'
df_customers = pd.read_csv(customers,
                             dtype= {
                                 'customer_id': 'str'
                             })

In [ ]:
df_customers.head()

In [ ]:
df_customers.info()

In [ ]:
products = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
df_products = pd.read_csv(products,
                             dtype= {
                                 'article_id': 'str'
                             })

In [ ]:
df_products.info()

In [ ]:
# embed_prod = nn.Embedding(len(df_products.index), len_embed)
embed_prod = torch.randn(len(df_products.index), len_embed)
embed_prod = embed_prod.to(device, dtype=torch.float)

In [ ]:
embed_prod.get_device()

In [ ]:
# embed_prod(torch.LongTensor([3]))

In [ ]:
# temp = embed_prod(torch.LongTensor([3,4])).squeeze()

In [ ]:
# temp

In [ ]:
# for elt in temp:
#     print(elt)

In [ ]:
# input1 = torch.randn(100, 128)
# input1

In [ ]:
# input2 = torch.randn(100, 128)
# cos = nn.CosineSimilarity(dim=1, eps=1e-6)
# output = cos(input1, input2)
# output

In [ ]:
date_split = '2020-09-15'
train = df_transactions[:date_split]
test = df_transactions[date_split:]
len(train), len(test)

In [ ]:
embed_custom = nn.Embedding(len(df_customers.index), len_embed)
embed_custom = embed_custom.to(device, dtype=torch.float)

In [ ]:
# embed_custom(torch.LongTensor([3])).squeeze()

In [ ]:
# first = embed_custom(torch.LongTensor([3]))
# with torch.no_grad():
#     second = first
# second

In [ ]:
# second.detach()

In [ ]:
# second

In [ ]:
# environment.py
# in this cell we build the environment class that simulate a RL environment

class Environment:
    def __init__(self, data_transactions, emb_customers, emb_products, df_customers, df_products):
        self.data = data_transactions
        self.emb_customers = emb_customers
        self.emb_prod = emb_products
        self.customers = df_customers
        self.products = df_products
        # self.reset()
        
    def reset(self):
        self.done = False
        self.current_pos = 0
        # we must return the informations about the current user (and the products bought by him)
        # let's go first with the current user by getting his id
        self.userid = self.data.iloc[self.current_pos, 0]
        # from the userid, we must get its corresponding index  in df_customers
        self.user_index = self.customers.index[self.customers['customer_id']==self.userid].tolist()[0]
        # now we can get the user embedding informations from self.emb_customers
        self.user_embed = self.emb_customers(torch.LongTensor([self.user_index]).to(device, dtype=torch.long))
        print(self.user_embed.get_device())
        # in this first version we are not going to consider the last products used by our customer
        return self.user_embed
    
    def step(self, act):
        # here act is a list of 12 (num_prod_to_rec) products to recommend by their indices.
        # For example, it could be [1, 2, 3, ..., 11, 12] 
        reward = 0
        # First, me must find the index of the actual product used by the customer in self.products
        # let's go first with the current product by getting his id
        self.product_id = self.data.iloc[self.current_pos, 1]
        # from the product_id, we must get its corresponding index in df_products
        self.prod_index = self.products.index[self.products['article_id']==self.product_id].tolist()[0]
        reward = self.compute_reward(act, self.prod_index)
        if (self.current_pos < len(self.data.index)):
#             if ((self.current_pos % episode_length)!=0):
            self.current_pos += 1
            if (self.current_pos < len(self.data.index)):
                self.userid = self.data.iloc[self.current_pos, 0]
                self.user_index = self.customers.index[self.customers['customer_id']==self.userid].tolist()[0]
                self.user_embed = self.emb_customers(torch.LongTensor([self.user_index]).to(device, dtype=torch.long))
                self.done = False
                return self.user_embed, reward, self.done
            else:
                self.done = True
                return self.user_embed, reward, self.done
    
    def compute_reward(self, proposed_products, actual_product):
        if (actual_product in proposed_products):
            # take the position where the matching occurs
            postion = proposed_products.index(actual_product)
            proposed_products = proposed_products[:position+1]
        embed_proposed = self.emb_prod[torch.LongTensor(proposed_products).to(device, dtype=torch.long)]
        embed_actual = self.emb_prod[torch.LongTensor([actual_product]).to(device, dtype=torch.long)]
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        output = cos(embed_proposed, embed_actual)
        coeffs = [math.pow(beta, t) for t in range(len(proposed_products))]
        # convert coeffs to tensor
        coeffs = torch.Tensor(coeffs).to(device, dtype=torch.float)
        # apply element wise multiplication to output
        output = torch.mul(output, coeffs)
        # we sum that and multiply by 100
        reward = torch.sum(output)
        reward = torch.mul(reward, scale)
        reward = reward.item()
        return reward

In [ ]:
# models.py
class Actor(torch.nn.Module):
    def __init__(self, n_input, n_weight_out, n_features_out, product_feats):
        # n_input: number of features that represents a customer
        # n_weight_out: number of products we want to recommend represent by their features
        # n_features_out: number of features of each vector of recommend products
        # product_feats: matrix of vector products. Each line as the same number of
        # features as n_features_out
        super(Actor, self).__init__()
        self.n_input = n_input
        self.n_weight_out = n_weight_out
        self.n_features_out = n_features_out
        self.feats = product_feats
        l1 = 4 * self.n_input
        l2 = 2 * l1
        self.l1 = nn.Linear(self.n_input, l1)
        self.l2 = nn.Linear(l1, l2)
        self.l3 = nn.Linear(l2, self.n_weight_out * self.n_features_out)
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        the_weights = x.view(-1, self.n_weight_out, self.n_features_out)
        # take the dot product between the_weights and self.feats
        product_trans = torch.transpose(self.feats, 0, 1)
        all_scalar_prods = torch.matmul(the_weights, product_trans)
        # output shape of previous operation : (1, n_weight_out, num_of_products)
        # find the indices with the highest scalar products
        argmax = torch.argmax(all_scalar_prods, dim=2)
        return argmax

class Critic(torch.nn.Module):
    def __init__(self, n_input, n_rec_prod):
        # n_input: number of features that represents a customer
        # n_rec_prod: number of products recommend by our actor
        super(Critic, self).__init__()
        self.n_input = n_input
        self.n_rec_prod = n_rec_prod
        l1 = 4*(self.n_input + self.n_rec_prod)
        l2 = l1
        l3 = 1
        self.l1 = nn.Linear(self.n_input + self.n_rec_prod, l1)
        self.l2 = nn.Linear(l1, l2)
        self.l3 = nn.Linear(l2, l3)
    def forward(self, x, act):
        x = torch.cat((x, act), -1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

class Model(torch.nn.Module):
    def __init__(self, actor, critic):
        super(Model, self).__init__()
        self.actor = actor
        self.critic = critic
    def forward(self, x):
        act = self.actor(x)
        crit = self.critic(x, act)
        return act, crit

In [ ]:
# Just for testing purpose, we generate product_feats
prod_feats = torch.rand(105542, 30)

In [ ]:
# train.py
actor_model = Actor(n_input=len_embed, n_weight_out=num_prod_to_rec, n_features_out=30, product_feats=embed_prod)
actor_model.to(device)
critic_model = Critic(n_input=len_embed, n_rec_prod=num_prod_to_rec)
critic_model.to(device)
model = Model(actor_model, critic_model)
model.to(device)
model2 = copy.deepcopy(model)
model2.to(device)
model2.load_state_dict(model.state_dict())

In [ ]:
sync_freq = 50
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
epochs = 1
losses = []
mem_size = 5000
batch_size = 256
replay = deque(maxlen=mem_size)
syn_freq = 500
gamma = 0.9
j=0

In [ ]:
for i in range(epochs):
    # start a new environment
    env = Environment(df_transactions, embed_custom, embed_prod, df_customers, df_products)
    state1 = env.reset()
    status = 1
    while(status==1):
        j += 1
#         print(j)
        act_, crit_ = model(state1.detach()) # act_ and crit_ are tensors
        # step the environment
        act = act_.view(-1).tolist()
        state2, reward, done = env.step(act)
        exp = (state1, reward, state2, done)
        replay.append(exp)
        state1 = state2
        
        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)
            state1_batch = torch.cat([s1 for (s1, r, s2, d) in minibatch])
            reward_batch = torch.Tensor([r for (s1, r, s2, d) in minibatch])
            reward_batch = reward_batch.to(device, dtype=torch.float)
            state2_batch = torch.cat([s2 for (s1, r, s2, d) in minibatch])
            done_batch = torch.Tensor([d for (s1, r, s2, d) in minibatch])
            done_batch = done_batch.to(device, dtype=torch.float)
            _, Q1 = model(state1_batch)
            with torch.no_grad():
                _, Q2 = model2(state2_batch.detach())
            Y = reward_batch + gamma * ((1 - done_batch) * Q2)
            X = Q1
            loss = loss_fn(X, Y.detach())
            clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            losses.append(loss.item)
            optimizer.step()
            
            if j % syn_freq == 0:
                print(j)
                model2.load_state_dict(model.state_dict())
        if done:
            status = 0
losses = np.array(losses)

In [ ]:
# models.py

In [ ]:
#config.py

In [ ]:
# model_dispatcher.py